In [10]:
import pandas as pd
import numpy as np
import holidays
import calendar
import datetime 
import dateutil.relativedelta
import pandas_datareader.data as web
import requests 
from bs4 import BeautifulSoup
import json
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

### DataFrame de base External 

- Pour voir commment ca marche le code, on vous propose d'utiliser external_data2, copie du external original, pour voir comment il va etre transformé.
- Pour voir les fonction du creation de external_data, on vous a mis a disposition des autres nom de fichier, il suffit juste d'enlever des commentaire les df.to_csv (1) et (2) qui sont en commentaire en bas et de faire run le code en commentaire. 

(Faites attention, le code pour la distance est assez longue, pour cette raison on utilise la librarie tqdm pour vous montrez combien de temps il reste)

In [11]:
ext = pd.read_csv("submissions/starting_kit/external_data2.csv")
ext.head()

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,...,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,...,16,16,11,19,6,26.0,0.00,3,NaN,129
1,2011-09-02,ATL,36,29,22,17,15,14,61,46,...,16,16,16,24,7,34.0,0.00,2,NaN,185
2,2011-09-03,ATL,35,29,23,17,16,14,64,47,...,16,16,16,19,7,26.0,0.00,4,NaN,147
3,2011-09-04,ATL,27,24,22,22,19,16,93,72,...,16,14,4,21,9,26.0,6.10,6,Rain,139
4,2011-09-05,ATL,26,24,22,23,22,20,94,91,...,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149


 Column that will show if we it's a holiday day + weekend

###### US_holidays and weekends

In [12]:
def getUSHolidays(df):
    
    """
    Fonction pour le weekend and pour les jours feries. 
    """
    
    us_holidays = holidays.US()
    holid = []
    date_values=df.Date.tolist()
    
    for date in date_values:
        rep = date in us_holidays
        if rep=='True':
            rep = 1
        else:
            rep = 0
        holid.append(rep)
        
    df['Holidays'] = holid
    
    print('getUSHolidays(): Finished')
    
    return df

def findDay(date): 
    
    year, month, day = (int(i) for i in date.replace("-"," ").split(' '))        
    dayNumber = calendar.weekday(year, month, day) 
    days =["Monday", "Tuesday", "Wednesday", "Thursday", 
                         "Friday", "Saturday", "Sunday"] 
    return (days[dayNumber]) 

def getWeekend(df):
    
    getUSHolidays(df)
    
    date_values=df.Date.tolist()

    Weekend = []
    
    for date in date_values:
        wkend = findDay(date)
        if wkend=='Saturday' or wkend=='Sunday':
            wkend = 1
        else:
            wkend = 0
        Weekend.append(wkend)
    
    df['Weekend'] = Weekend
        
    print('getWeekend(): Finished')
    
    return df


def nonBusinessDay(df):
    
    getWeekend(df)
    
    df['N_BD'] = df.Holidays + df.Weekend
    for i in df['N_BD']:
        if i == 2:
            i = 1
    df.drop(columns=['Holidays', 'Weekend'], inplace = True)
    
    print('nonBusinessDay(): Finished')
    
    return df

###### Oil price lagged 1, 3, 6 and 9 mois. 

In [13]:
oil = web.DataReader("MJFUELUSGULF", "fred", '2008-01-01', '2015-01-01')
oil.reset_index(inplace=True)
oil["DATE"] = oil["DATE"].apply(lambda x: x.strftime('%Y-%m'))

def getOilprice(df,oil):
    """
    Fonction pour le prix du kerosene avec un lag de 1 ,3, 6, 12 mois.
    """
    
    date_values = df.Date.values
    offsetList = [1,3,6,12]
    
    for lag in tqdm(offsetList):
        DateOffSet = []
        
        for dat in date_values:
            
            dat = datetime.datetime.strptime(dat, "%Y-%m-%d")
            dateoff = str((dat - dateutil.relativedelta.relativedelta(months=lag)).date())
            DateOffSet.append(dateoff)
            
        df["DateOffSet"+str(lag)] = DateOffSet
        df["DateOffSet"+str(lag)] = pd.to_datetime(df["DateOffSet"+str(lag)])
        df["DateOffSet"+str(lag)] = df["DateOffSet"+str(lag)].apply(lambda x: x.strftime("%Y-%m"))
        oil.columns = ["DateOffSet"+str(lag), "Oil_Poffset"+str(lag)]
        df = pd.merge(df, oil, on="DateOffSet"+str(lag))
        df.drop(columns=["DateOffSet"+str(lag)], inplace = True)
    
    print('getOilprice(): Finished')
    
    return df

 ###### Get the City from the airport 

In [14]:
def getCity(df):
    
    """
    Fonction qui prends la ville a partir de l'aeroport.
    """
    
    City = []

    airp=df.AirPort.unique().tolist()
    
    for iata in tqdm(airp):
        url = 'https://www.airnav.com/airport/'+str(iata)
        page = requests.get(url, verify=False)
        soup = BeautifulSoup(page._content, 'html.parser')
        table = soup.findAll('font',{'face': 'Verdana, Arial, Helvetica, sans-serif'})
        city = (str(table[1])).split('br/>')[1].split(',')[0]
        City.append(city)
        
    City = list(map(lambda st: str.replace(st, "Dallas-Fort Worth", "Dallas"), City)) 
    DictCity = dict(zip(airp, City))
    
    df['City'] = df['AirPort'].map(DictCity)
    
    print('getCity(): Finished')
    
    return df

In [15]:
def get_city_opendata(city, country):
    tmp = 'https://public.opendatasoft.com/api/records/1.0/search/?dataset=worldcitiespop&q=%s&sort=population&facet=country&refine.country=%s'
    cmd = tmp % (city, country)
    res = requests.get(cmd)
    dct = json.loads(res.content)
    out = dct['records'][0]['fields']
    return out


def getPopulation(df):
    
    """
    Fonction qui prends la population a partir de la Ville et du Pays grace au database du site.
    """
    
    population = []
    ville =df.City.unique().tolist()
    
    for cit in tqdm(ville):
        pop = get_city_opendata(cit, 'us')
        population.append(pop['population'])
        
    DictPop =  dict(zip(ville, population))
    df['Population'] = df['City'].map(DictPop)
    
    print('getPopulation(): Finished')
    
    return df

Comme expliquer dans le Word, le but c'etait de creer dans le meme csv deux tables differentes et le tenir formatter dans cette maniere la en diagonal. La fonction Bigdataframe(df): creer la grande table a gauche. 

In [16]:
def Bigdataframe(df):
    """
    Fonction qui appele tout les fonctions en haut. 
    """
    
    df = nonBusinessDay(df)
    df = getOilprice(df,oil)
    df = getCity(df)
    df = getPopulation(df)
    
    return df

ext = Bigdataframe(ext)

getUSHolidays(): Finished
getWeekend(): Finished
nonBusinessDay(): Finished


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.01s/it]


getOilprice(): Finished


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:53<00:00,  2.10s/it]


getCity(): Finished


100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [00:07<00:00,  2.40it/s]


getPopulation(): Finished


In [19]:
# ext.to_csv('submissions/starting_kit/support1.csv', index=False)
print(ext.shape)
ext.head(1)

(11040, 31)


,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,...,CloudCover,Events,WindDirDegrees,N_BD,Oil_Poffset1,Oil_Poffset3,Oil_Poffset6,Oil_Poffset12,City,Population
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,...,3,NaN,129,0,3.008,3.046,3.125,2.114,Atlanta,422908


###### DataFrame pour la Distance

Creation de l'autre DataFrame, celui de droite. Il va contenir que trois colonne. 

In [236]:
def getDistance(df):
    
    """
    Fonction qui fait tout les combinaison posible entre les distance et trouve la distance entre les aeroport.
    
    """

    distance = []
    CityDep = []
    CityArr = []

    Airp1=df.AirPort.unique().tolist()
    Airp2=df.AirPort.unique().tolist()

    for i in  tqdm(Airp1):
        for j in Airp2: 
            CityDep.append(i)
            CityArr.append(j)
            
            if i==j:
            
                distance.append(0)
    
            else:
                url = vfrom-'+str(i).lower()+'/to-'+str(j).lower()+'/'
                page = requests.get(url, verify=False)
                soup = BeautifulSoup(page.content, 'html.parser')
                table = soup.findAll('div',{'class': 'highlight-grey pad-large full-width item-block'})
                table=str(table)
                km=table.split('km')[0].split('miles')[1].replace("/","").strip()
                distance.append(km)   
        
    #DictPop =  dict(zip(CityDep,zip(CityArr, distance)))
    
    return CityDep,CityArr, distance

CityDep,CityArr, distance = getDistance(ext)


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [03:38<00:00, 11.22s/it]

In [293]:
extDataDistance = pd.DataFrame(CityDep,columns =['Departure'])
extDataDistance['Arrival'] = CityArr
extDataDistance['Distance'] = distance
# extDataDistance.to_csv('/submissions/starting_kit/support2.csv', index=False)
extDataDistance.head()

,Departure,Arrival,Distance
0,ATL,ATL,0
1,ATL,DEN,1925.48
2,ATL,JFK,1223.36
3,ATL,LAS,2803.98
4,ATL,ORD,975.64


### Creation d'un DataFrame commun et creation du fichier external_data.csv

Cette partie, prends les deux csv file creer separement, et le reunis dans un, comme d'ecrit dans le Word.
Elle desactivé pour securité, et le csv des autres tables sont supprimer aussi pour securité.
Mais vous pouvez utiliser cette technique si vous voulez utiliser le meme csv avec differentes tables. 

In [304]:
# import os
# import glob
# #set working directory
# os.chdir("submissions/starting_kit")

# #find all csv files in the folder
# #use glob pattern matching -> extension = 'csv'
# #save result in list -> all_filenames
# extension = 'csv'
# all_filenames = [i for i in glob.glob('support*.{}'.format(extension))]
# #print(all_filenames)

# #combine all files in the list
# combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
# #export to csv
# combined_csv.to_csv("external_dataTest.csv",index = False)

### Importation et creation des deux tables differentes 

Si vous voulez indentifiez les tables, vous pouvez a partir de cette partie en bas. Vous allez resuir a recuperer deux tables a partir du meme csv. 

In [20]:
df = pd.read_csv("submissions/starting_kit/external_data.csv")

distanceDF = pd.DataFrame()
distanceDF = df[['Departure','Arrival','Distance']] 
distanceDF.dropna(inplace = True)
distanceDF.reset_index(inplace=True,drop= True)

dataEXT = df.drop(['Departure','Arrival','Distance'], axis=1)
dataEXT.rename(columns={"Date": "DateOfDeparture"})
dataEXT.dropna(how='all',inplace=True)

In [21]:
distanceDF.head()

,Departure,Arrival,Distance
0,ATL,ATL,0.00
1,ATL,DEN,1925.48
2,ATL,JFK,1223.36
3,ATL,LAS,2803.98
4,ATL,ORD,975.64


In [22]:
dataEXT.head(1)

,AirPort,City,CloudCover,Date,Dew PointC,Mean Humidity,Mean Sea Level PressurehPa,Mean TemperatureC,Mean VisibilityKm,Mean Wind SpeedKm/h,MeanDew PointC,N_BD,Oil_Poffset1,Oil_Poffset12,Oil_Poffset3,Oil_Poffset6,Population,WindDirDegrees
0,ATL,Atlanta,3.0,2011-09-01,21.0,56.0,1019.0,29.0,16.0,6.0,18.0,0.0,3.008,2.114,3.046,3.125,422908.0,129.0
